Analysing with Decision tree
Setup: put it in the same folder as "unifed_csv_without_duplicated_company.csv" and "ROI.csv" (generated by get_roi notebook) 

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split   
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier 
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from IPython.display import Image
import pydotplus
import graphviz
from sklearn.model_selection import GridSearchCV

# Optional
# import library for visuaiization
from sklearn import tree
import matplotlib.pyplot as plt
from collections import Counter

In [17]:
#Read both CSV
df=pd.read_csv("unifed_csv_without_duplicated_company.csv")
df.info(max_cols=1000)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37198 entries, 0 to 37197
Data columns (total 132 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   company_uuid                      37198 non-null  object 
 1   country_code                      33609 non-null  object 
 2   region                            33609 non-null  object 
 3   city                              33609 non-null  object 
 4   operating                         37198 non-null  int64  
 5   closed                            37198 non-null  int64  
 6   success                           37198 non-null  int64  
 7   success_acquired                  37198 non-null  int64  
 8   success_ipo                       37198 non-null  int64  
 9   num_funding_rounds                12036 non-null  float64
 10  total_funding_usd                 9093 non-null   float64
 11  founded_on                        37198 non-null  float64
 12  las

In [28]:
#Create dataframe for features
features=df[["company_uuid", "operating","closed","success","success_acquired","success_ipo", "num_funding_rounds","Financial Services","FinTech","Finance","Blockchain", "Information Technology", "Software",  "Payments" , "Cryptocurrency", "Venture Capital","Internet", "Banking", "Consulting",  "Mobile Payments","E-Commerce", "Insurance", "Bitcoin", "Artificial Intelligence", "Mobile", "Crowdfunding","SaaS",  "Real Estate", "Apps", "Personal Finance","Accounting", "Mobile Apps", "Asset Management", "Marketplace", "Lending", "Big Data", "Machine Learning", "gender_male", "gender_female" ]].copy()
dummy= df.columns[70:]
dummy_df= df[dummy]
features= features.join(dummy_df)
features.info(max_cols=1000)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37198 entries, 0 to 37197
Data columns (total 101 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   company_uuid                     37198 non-null  object 
 1   operating                        37198 non-null  int64  
 2   closed                           37198 non-null  int64  
 3   success                          37198 non-null  int64  
 4   success_acquired                 37198 non-null  int64  
 5   success_ipo                      37198 non-null  int64  
 6   num_funding_rounds               12036 non-null  float64
 7   Financial Services               37198 non-null  int64  
 8   FinTech                          37198 non-null  int64  
 9   Finance                          37198 non-null  int64  
 10  Blockchain                       37198 non-null  int64  
 11  Information Technology           37198 non-null  int64  
 12  Software         

In [29]:
features= features.drop(columns=["time_in_year","investment_delta", "norm_amount_delta", "momentum", "max_norm_amount_delta", "roi_from_mean_momentum", "continent", "ROI" ])

In [30]:
for i in features.columns: 
    x= features[i].value_counts()
    print("column: ",i,", value is:",x)
    print()

column:  company_uuid , value is: 1674d3c9-c7a8-447d-a77f-c1cb2455bd48    1
89497b76-5f7c-4283-aa37-f8dcc9a5941a    1
c691a9a4-97a8-4674-864e-c28352245852    1
52cd82f6-9afe-4a4c-920b-e92776ea55e2    1
c16edeb7-31bf-ad1b-cde0-6c43a099bc1c    1
                                       ..
5924f1d0-2e21-4794-87bb-f54268939b05    1
6afb6eb4-ac02-1a91-6829-4bb7c6ccdb1f    1
192acc26-46d6-5961-2a18-2ec462242091    1
3a36dd8b-5a8e-4d10-8930-34e52dd9964d    1
55019c3e-e76a-49e2-9258-3a19227b6257    1
Name: company_uuid, Length: 37198, dtype: int64

column:  operating , value is: 1    34671
0     2527
Name: operating, dtype: int64

column:  closed , value is: 0    36130
1     1068
Name: closed, dtype: int64

column:  success , value is: 0    35739
1     1459
Name: success, dtype: int64

column:  success_acquired , value is: 0    36071
1     1127
Name: success_acquired, dtype: int64

column:  success_ipo , value is: 0    36866
1      332
Name: success_ipo, dtype: int64

column:  num_funding_rounds

In [31]:
#Deal with null value in num_funding_rounds 
features["num_funding_rounds"]=features["num_funding_rounds"].fillna(0)

In [32]:
# Join with ROI.csv from ROI.ipynb
features["mean_norm_amount_delta"]=features["mean_norm_amount_delta"].fillna(0)
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37198 entries, 0 to 37197
Data columns (total 93 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   company_uuid                     37198 non-null  object 
 1   operating                        37198 non-null  int64  
 2   closed                           37198 non-null  int64  
 3   success                          37198 non-null  int64  
 4   success_acquired                 37198 non-null  int64  
 5   success_ipo                      37198 non-null  int64  
 6   num_funding_rounds               37198 non-null  float64
 7   Financial Services               37198 non-null  int64  
 8   FinTech                          37198 non-null  int64  
 9   Finance                          37198 non-null  int64  
 10  Blockchain                       37198 non-null  int64  
 11  Information Technology           37198 non-null  int64  
 12  Software          

In [35]:
# define independent variables/attirbutes/features
x = features.drop(columns=['mean_norm_amount_delta', "company_uuid"])
y = features["mean_norm_amount_delta"]

#Make "mean_norm_amount_delta" column 1/0
binary_labels = [0,1]
y = pd.qcut(y, 2, labels = binary_labels)
y

0        0
1        0
2        0
3        0
4        0
        ..
37193    0
37194    0
37195    0
37196    0
37197    1
Name: mean_norm_amount_delta, Length: 37198, dtype: category
Categories (2, int64): [0 < 1]

In [36]:
# Implement Decision tree
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=5)
model=DecisionTreeClassifier()
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [37]:
# use the trained model to predict test set for evaluation
pred_test = model.predict(X_test)

# print out evaluation result
print("Accuracy:{}".format(accuracy_score(y_test, pred_test, normalize=True, sample_weight=None)))
print("Classification Report:\n{}".format(classification_report(y_test, pred_test)))
print("Confusion Matrix:\n{}".format(confusion_matrix(y_test, pred_test)))

Accuracy:0.942741935483871
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.97      0.97     10303
           1       0.64      0.57      0.61       857

    accuracy                           0.94     11160
   macro avg       0.80      0.77      0.79     11160
weighted avg       0.94      0.94      0.94     11160

Confusion Matrix:
[[10031   272]
 [  367   490]]
